In [1]:
from kgx import Transformer, NeoTransformer

/Users/deepak.unni3/GIT/kgx/test/lib/python3.6/site-packages/cachier/mongo_core.py:24: UserWarning: Cachier warning: pymongo was not found. MongoDB cores will not work.
  "Cachier warning: pymongo was not found. MongoDB cores will not work.")


In [2]:
# Monarch-Lite

# Credentials can be found from 'Registry of Biolink-compatible Neo4 instances' spreadsheet
monarch_uri = ''
monarch_username = ''
monarch_password = ''

# Initialize NeoTransformer
monarch_lite_transformer = NeoTransformer(uri=monarch_uri, username=monarch_username, password=monarch_password)

# Define filters
monarch_lite_transformer.set_filter("subject_category", "gene")
monarch_lite_transformer.set_filter("object_category", "phenotypic_feature")
monarch_lite_transformer.set_filter("edge_label", "has_phenotype")

# Load nodes and edges from remote Monarch-Lite Neo4j instance
start = 0
end = 20000
monarch_lite_transformer.load(start=start, end=end)
monarch_lite_transformer.report()

2018-06-08 23:46:57,499 root       DEBUG      MATCH (n:gene:phenotypic_feature ) RETURN n SKIP 0 LIMIT 1000;
2018-06-08 23:46:57,719 root       DEBUG      MATCH (s:gene)-[p:has_phenotype ]->(o:phenotypic_feature)
            RETURN s,p,o
            SKIP 0 LIMIT 1000;
2018-06-08 23:47:00,432 root       DEBUG      MATCH (s:gene)-[p:has_phenotype ]->(o:phenotypic_feature)
            RETURN s,p,o
            SKIP 1000 LIMIT 1000;
2018-06-08 23:47:02,821 root       DEBUG      MATCH (s:gene)-[p:has_phenotype ]->(o:phenotypic_feature)
            RETURN s,p,o
            SKIP 2000 LIMIT 1000;
2018-06-08 23:47:08,770 root       DEBUG      MATCH (s:gene)-[p:has_phenotype ]->(o:phenotypic_feature)
            RETURN s,p,o
            SKIP 3000 LIMIT 1000;
2018-06-08 23:47:13,840 root       DEBUG      MATCH (s:gene)-[p:has_phenotype ]->(o:phenotypic_feature)
            RETURN s,p,o
            SKIP 4000 LIMIT 1000;
2018-06-08 23:47:21,343 root       DEBUG      MATCH (s:gene)-[p:has_phenotype ]

In [3]:
# RTX

# Credentials can be found from 'Registry of Biolink-compatible Neo4 instances' spreadsheet
rtx_uri = ''
rtx_username = ''
rtx_password = ''

# Initialize NeoTransformer
rtx_transformer = NeoTransformer(uri=rtx_uri, username=rtx_username, password=rtx_password)

# Define filters
rtx_transformer.set_filter("subject_category", "phenotypic_feature")
rtx_transformer.set_filter("object_category", "anatomical_entity")
rtx_transformer.set_filter("edge_label", "affects")

# Load nodes and edges from remote SemMedDB Neo4j instance
start = 0
end = 20000
rtx_transformer.load(start=start, end=end)
rtx_transformer.report()

2018-06-08 23:48:01,666 root       DEBUG      MATCH (n:phenotypic_feature:anatomical_entity ) RETURN n SKIP 0 LIMIT 1000;
2018-06-08 23:48:02,108 root       DEBUG      MATCH (s:phenotypic_feature)-[p:affects ]->(o:anatomical_entity)
            RETURN s,p,o
            SKIP 0 LIMIT 1000;
2018-06-08 23:48:03,641 root       DEBUG      MATCH (s:phenotypic_feature)-[p:affects ]->(o:anatomical_entity)
            RETURN s,p,o
            SKIP 1000 LIMIT 1000;
2018-06-08 23:48:04,893 root       DEBUG      MATCH (s:phenotypic_feature)-[p:affects ]->(o:anatomical_entity)
            RETURN s,p,o
            SKIP 2000 LIMIT 1000;
2018-06-08 23:48:06,144 root       DEBUG      MATCH (s:phenotypic_feature)-[p:affects ]->(o:anatomical_entity)
            RETURN s,p,o
            SKIP 3000 LIMIT 1000;
Total number of nodes: 3856
Total number of edges: 2963


In [7]:
# Then we merge Monarch-Lite graph with RTX graph
graphs = [monarch_lite_transformer.graph, rtx_transformer.graph]
merged_transformer = Transformer()
merged_transformer.merge(graphs)

In [8]:
# Now, we get a phenotypic_entity common in both the graphs
short_ear_node = [n for n in merged_transformer.graph.nodes(data=True) if n[1]['id'] == 'HP:0400005'][0]
short_ear_node

('HP:0400005',
 {'expanded': True,
  'name': 'Short ear',
  'rtx_name': 'HP:0400005',
  'description': 'Median longitudinal ear length less than two SD above the mean determined by the maximal distance from the superior aspect to the inferior aspect of the external ear.',
  'id': 'HP:0400005',
  'accession': '0400005',
  'UUID': 'ec2e0018-5907-11e8-95d6-060473434358',
  'category': 'phenotypic_feature',
  'uri': 'http://purl.obolibrary.org/obo/HP_0400005',
  'seed_node_uuid': 'dbf1734c-5907-11e8-95d6-060473434358',
  'labels': ['phenotypic_feature', 'Base']})

In [13]:
# we get all in-coming edges to this node
in_edges = merged_transformer.graph.in_edges(short_ear_node[0], data=True)

# we can see that there are 8 incoming edges from gene nodes (from Monarch-Lite)
in_edges

[('HGNC:13315',
  'HP:0400005',
  {'is_defined_by': 'Monarch',
   'provided_by': ['https://data.monarchinitiative.org/ttl/hpoa.ttl'],
   'evidence': ['http://purl.obolibrary.org/obo/ECO_0000501'],
   'qualifier': 'direct',
   'publications': [],
   'relation': 'RO:0002200',
   'subject': 'HGNC:13315',
   'object': 'HP:0400005',
   'id': 3397073,
   'type': 'has_phenotype',
   'predicate': 'has_phenotype'}),
 ('HGNC:25726',
  'HP:0400005',
  {'is_defined_by': 'Monarch',
   'provided_by': ['https://data.monarchinitiative.org/ttl/hpoa.ttl'],
   'evidence': ['http://purl.obolibrary.org/obo/ECO_0000322',
    'http://purl.obolibrary.org/obo/ECO_0000220',
    'http://purl.obolibrary.org/obo/ECO_0000501'],
   'qualifier': 'direct',
   'publications': ['http://www.ncbi.nlm.nih.gov/pubmed/25644381',
    'http://www.ncbi.nlm.nih.gov/pubmed/1746601'],
   'relation': 'RO:0002200',
   'subject': 'HGNC:25726',
   'object': 'HP:0400005',
   'id': 3904995,
   'type': 'has_phenotype',
   'predicate': 'h

In [14]:
# we get all out-going edges from this node
out_edges = merged_transformer.graph.out_edges(short_ear_node[0], data=True)

# we can see that there is 1 outgoing edge from our node to anatomical_entity (from RTX)
out_edges

[('HP:0400005',
  'UBERON:0001691',
  {'is_defined_by': 'RTX',
   'predicate': 'affects',
   'source_node_uuid': 'ec2e0018-5907-11e8-95d6-060473434358',
   'target_node_uuid': '0a42cd6a-5af5-11e8-95d6-060473434358',
   'provided_by': 'BioLink',
   'seed_node_uuid': 'dbf1734c-5907-11e8-95d6-060473434358',
   'relation': 'affects',
   'subject': 'HP:0400005',
   'object': 'UBERON:0001691',
   'id': 16269033,
   'type': 'affects'})]